In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split


In [3]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/iseongmin/.surprise_data/ml-100k


In [4]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [5]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.846363949593691, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [8]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

In [10]:
import pandas as pd

ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
ratings.to_csv('./data/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [13]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./data/ml-latest-small/ratings_noh.csv', reader=reader)

In [15]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [17]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [18]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8735  0.8688  0.8755  0.8752  0.8736  0.8733  0.0024  
MAE (testset)     0.6693  0.6648  0.6730  0.6740  0.6738  0.6710  0.0035  
Fit time          0.45    0.43    0.41    0.41    0.44    0.43    0.01    
Test time         0.05    0.05    0.05    0.13    0.05    0.07    0.03    


{'test_rmse': array([0.87350708, 0.86884929, 0.87551811, 0.87517016, 0.87360987]),
 'test_mae': array([0.66927945, 0.66482588, 0.67299368, 0.67395208, 0.6737829 ]),
 'fit_time': (0.45131897926330566,
  0.4250369071960449,
  0.41326904296875,
  0.41201305389404297,
  0.43528223037719727),
 'test_time': (0.05206704139709473,
  0.05243206024169922,
  0.05233001708984375,
  0.13451790809631348,
  0.05308413505554199)}

In [19]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.877564156045655
{'n_epochs': 20, 'n_factors': 50}


In [20]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file='./data/ml-latest-small/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [23]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [27]:
movies = pd.read_csv('./data/ml-latest-small/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [28]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [29]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수 :', len(unseen_movies), '전체 영화 수:', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수 : 9696 전체 영화 수: 9742


In [44]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)

    top_predictions = predictions[:top_n]

    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_title = movies[movies['movieId'].isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_title, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movies_preds = recomm_movie_by_surprise(algo, 9, unseen_movies)

print('### Top-20 추천 영화 리스트 ###')
for top_movie in top_movies_preds:
    print(top_movie[1], ':', top_movie[2])


평점 매긴 영화 수: 46 추천 대상 영화 수 : 9696 전체 영화 수: 9742
### Top-20 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


In [37]:
top_movies_preds

[(858, 'Seven (a.k.a. Se7en) (1995)', 4),
 (260, 'Usual Suspects, The (1995)', 4),
 (296, 'Star Wars: Episode IV - A New Hope (1977)', 4),
 (1196, 'Pulp Fiction (1994)', 4),
 (50, 'Shawshank Redemption, The (1994)', 4),
 (1104, 'Silence of the Lambs, The (1991)', 4),
 (1210, 'Ghost in the Shell (Kôkaku kidôtai) (1995)', 4),
 (1213, 'Godfather, The (1972)', 4),
 (1242, 'Streetcar Named Desire, A (1951)', 4),
 (593, 'Star Wars: Episode V - The Empire Strikes Back (1980)', 4),
 (47, 'Brazil (1985)', 4),
 (1233, 'Star Wars: Episode VI - Return of the Jedi (1983)', 4),
 (56782, 'Goodfellas (1990)', 4),
 (741, 'Boot, Das (Boat, The) (1981)', 4),
 (318, 'Glory (1989)', 4),
 (2324, 'Indiana Jones and the Last Crusade (1989)', 4),
 (6787, 'Life Is Beautiful (La Vita è bella) (1997)', 4),
 (58559, "All the President's Men (1976)", 4),
 (1291, 'There Will Be Blood (2007)', 4),
 (1199, 'Dark Knight, The (2008)', 4)]